In [ ]:
READ ME

1. mariaDB와 PHP가 필요합니다. 없으시다면 돌아가셔서 mariaDB_php_completed.ipynb를 실행하세요.
2. 1번이 완료 되었으면, 터미널에 들어가셔서 mariaDB를 입력하시고

CREATE USER 'for_remote_jupyter'@'주피터 서버의 ip입력' IDENTIFIED BY 'jupyter';
GRANT ALL PRIVILEGES ON *.* TO 'for_remote_jupyter'@'주피터 서버의 ip입력' WITH GRANT OPTION;
FLUSH PRIVILEGES;
SELECT User, Host FROM mysql.user; --로 확인--


위의 내용을 입력해주세요.

3. DNS가 필요합니다. 없으신 경우에 돌아가셔서 dns.ipynb의 실행부로 가셔서 init()함수를 실행하세요.

4. DNS로 설정만 하시는 경우에 dns.ipynb의 mod_zone() 함수를 실행하세요.
   init()함수는 실행하실 필요가 없습니다. 
   따라서 init() 함수는 주석하시고 mod_zone()함수만 실행하세요.

5. 그 후에 '나' 셸을 실행하시고, '가' 셸을 실행하세요

6. 터미널로 돌아가셔서, systemctl restart httpd; systemctl restart named 해주세용.


In [23]:
#######
# 가  #
#######


##################################################################################
#                      < 주피터 서버와 원격 DB 서버가 다른 경우 >                     #
#  주피터 서버 (172.16.16.100) , 원격 DB 서버(172.16.16.25)                         #
#  1. 주피터 서버에서 원격 DB 서버로 접근합니다.                                       #
#  2. 이를 위해선 원격 DB에서는 'example'@'172.16.16.100'으로 계정을 만들어주셔야해요.   #
#     password는 asd123!@    ** 이 내용은 터미널에서 직접 입력해야합니다                #  
#  3. 그 후에는 위에서 만든 계정을 바탕으로 pymysql.connect로 커넥터를 생성합니다.        #
#     이때 건네주는 host와 id는 위에서 만든 '172.16.16.100','example'입니다.           #
#     그리고 password는 asd123!@입니다.                                             #
#     이제 주피터 서버에서 원격 DB 서버로 DB 명령이 가능해집니다.                        #
#  4. 이제 원격 DB 서버에서 계정을 만듭니다.                                           #
#     이때에는 'example1'@'172.16.16.25'로 ip가 DB서버입니다.                         #
###################################################################################
import pymysql

db_server_ip = "172.16.16.25"  ### DB서버의 ip입력
port = "3306"
id_ = "for_remote_jupyter"
pw = "jupyter"

## 3.
db = pymysql.connect(host=db_server_ip, user=id_,passwd=pw,charset='utf8') # 커넥터 생성
cur = db.cursor() # cursor : 접속한 상태를 가리킴. 이제 그 뒤의 내용은 모두 원격에서 실행됩니다.

cmd = "SELECT User, Host FROM mysql.user" # DBMS에는 ; 반드시 붙여야하지만 자동으로 여기선 붙여줌
cur.execute(cmd) # 배치 방식 


db_name = input("만들 db 이름을 입력해주세요. ex) test_db : ")
user_name = input("만들 db 유저의 이름을 입력해주세요 ex) test_user : ")
user_pw = input("password를 입력해주세요 ex. test_passwd : ")
mk_user_list = [
        f"CREATE DATABASE {db_name}",
        f"CREATE USER '{user_name}'@'{db_server_ip}' IDENTIFIED BY '{user_pw}'",
        f"GRANT ALL PRIVILEGES ON {db_name}.* TO '{user_name}'@'{db_server_ip}'",
        "FLUSH PRIVILEGES"
]

for i in mk_user_list :
    cur.execute(i)


rows = cur.fetchall() # DMS에게 commit 전송하고 그 후에 반환되는 값을 rows에 초기화한다.

for i in rows:
    print(i)

cur.close()

만들 db 이름을 입력해주세요. ex) test_db :  test_db
만들 db 유저의 이름을 입력해주세요 ex) test_user :  test_user
password를 입력해주세요 ex. test_passwd :  test_passwd


In [22]:
#######
# 나  #
#######

import paramiko
import time
import sys

user = "root"
pw = "asd123!@"
ip = "172.16.16.25"
password = "asd123!@"

vhost_name = "vhost.conf" # default는 vhost.conf입니다.
def cc(cmd):
    cli = paramiko.SSHClient()
    cli.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    cli.connect(ip, username=user, password=pw, port=22)
    stdin, stdout, stderr = cli.exec_command(cmd, get_pty=True)  # ← 여기가 핵심
    out = stdout.read().decode()
    err = stderr.read().decode()
    print(out if out else err)
    cli.close()

def cc_return(cmd):
    cli = paramiko.SSHClient()
    cli.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    cli.connect(ip, username=user, password=pw, port=22)
    stdin, stdout, stderr = cli.exec_command(cmd, get_pty=True)  # ← 여기가 핵심
    out = stdout.read().decode()
    err = stderr.read().decode()
    return out if out else err

def install_prev_packages():
    cc("dnf -y install wget")
    cc("dnf -y install unzip")
def install_and_move():
    commands = """
cd ~
wget https://ko.wordpress.org/latest-ko_KR.zip
mkdir wp
mv latest-ko_KR.zip ./wp
cd wp
unzip latest-ko_KR.zip
cd wordpress
useradd wp
mv * /home/wp
cd /home
chmod 755 -R wp
chown -R wp:wp wp
    """
    cc(commands)

def set_wp_config_php():
    db_name = input("만들 db 이름을 입력해주세요. ex) test_db : ")
    user_name = input("만들 db 유저의 이름을 입력해주세요 ex) test_user : ")
    user_pw = input("password를 입력해주세요 ex. test_passwd : ")
    commands = f"""
cd /home/wp
mv wp-config-sample.php wp-config.php
sed -i "s|define( 'DB_NAME', 'database_name_here' );|define( 'DB_NAME', '{db_name}' );|" wp-config.php
sed -i "s|define( 'DB_USER', 'username_here' );|define( 'DB_USER', '{user_name}' );|" wp-config.php
sed -i "s|define( 'DB_PASSWORD', 'password_here' );|define( 'DB_PASSWORD', '{user_pw}' );|" wp-config.php
sed -i "s|define( 'DB_HOST', 'localhost' );|define( 'DB_HOST', '{ip}' );|" wp-config.php
"""
    cc(commands)

def allow_mariadb_at_remote():
    mariadb_check = input("mariadb와 php가 존재해야합니다 있으면 yes를 입력해주세요.")
    if(mariadb_check != "yes") :
        return
    commands = """
cat << EOF >> /etc/my.cnf
[mysqld]
bind-address = 0.0.0.0 
EOF
systemctl restart mariadb

"""
    cc(commands)
def set_http():
    global vhost_name;
    http_check = input("httpd가 있어야합니다. 있으면 yes를 입력해주세요.")
    if(http_check != "yes"):
        print("lamp_ftp_completed.ipynb을 실행하세요.")
        return 
    dns_check = input("dns가 있어야합니다. 있으면 yes를 입력해주세요.")
    if(http_check != "yes"):
        print("dns.ipynb을 실행하세요.")
        return 
    cc("ls -l /etc/httpd/conf.d")
    num = cc_return("ls /etc/httpd/conf.d | wc -l").strip()
    if(int(num) > 5) :
        print("현재 vhost 파일이 두개이상 있습니다")    
        print("README, autoindex.conf, userdir.conf, welcome.conf외에 다른 vhost 파일들을 확인하시고")
        print("lamp_ftp_completed.ipynb의 READE ME를 읽고 알맞게 고쳐주세요.")

    vhost_name = input("앞에서 확인하셨던 사용하실 zone file을 입력해주세요. ex vhost.conf")

def set_vhost() :
    print("vhost의 내용을 수정하는 작업입니다.")
    VHOST_CONF = f"/etc/httpd/conf.d/{vhost_name}"
   
    while(True) :
        print(f"현재 {VHOST_CONF}의 내용은 아래와 같습니다")
        cc(f"cat {VHOST_CONF}")
    
        num = int(input("(1) vhost 추가하기, (0) vhost 파일 수정 종료"))
        if( num == 1 ) : 
            SERVER_NAME = input("ServerName을 입력하세요. 주의!! 나중에 dns를 만들때 도메인 네임과 일치시키지 않으면 문제발생 (예: wp.test.com): ")
            cmd = f"""
cat <<EOF >> "{VHOST_CONF}"
<VirtualHost *:80>
    ServerAdmin webmaster@localhost
    ServerName {SERVER_NAME}
    DocumentRoot "/home/wp/"
    
    <Directory "/home/wp/">
        AllowOverride None
        Options Indexes FollowSymLinks
        Require all granted
    </Directory>

</VirtualHost>
EOF
"""
            cc(cmd)
        if(num == 0) : break;
    
    print(f" {VHOST_CONF} 생성 완료 및 httpd 및 named 재시작")
    cc("systemctl restart httpd")
    cc("systemctl restart named")
def wp_in():
    install_prev_packages()
    install_and_move()
    set_wp_config_php()
    allow_mariadb_at_remote()
    set_http()
    set_vhost()

wp_in()
    
    

Last metadata expiration check: 0:54:00 ago on Thu Nov 13 16:12:09 2025.
Package wget-1.21.1-8.el9_4.x86_64 is already installed.
Dependencies resolved.
Nothing to do.
Complete!

Last metadata expiration check: 0:54:01 ago on Thu Nov 13 16:12:09 2025.
Dependencies resolved.
 Package         Architecture     Version                Repository        Size
Installing:
 unzip           x86_64           6.0-58.el9_5           baseos           180 k

Transaction Summary
Install  1 Package

Total download size: 180 k
Installed size: 390 k
unzip-6.0-58.el9_5.x86_64.rpm                   1.7 MB/s | 180 kB     00:00    
--------------------------------------------------------------------------------
Total                                           266 kB/s | 180 kB     00:00     
Running transaction check
Transaction check succeeded.
Running transaction test
Transaction test succeeded.
Running transaction
  Preparing        :                                                        1/1 
  Installin

만들 db 이름을 입력해주세요. ex) test_db :  test_db
만들 db 유저의 이름을 입력해주세요 ex) test_user :  test_user
password를 입력해주세요 ex. test_passwd :  test_passwd


mariadb와 php가 존재해야합니다 있으면 yes를 입력해주세요. yes


httpd가 있어야합니다. 있으면 yes를 입력해주세요. yes
dns가 있어야합니다. 있으면 yes를 입력해주세요. yes


total 24
-rw-r--r--. 1 root root  400 Sep  3 23:59 README
-rw-r--r--. 1 root root 2916 Sep  3 23:59 autoindex.conf
-rw-r--r--  1 root root 1668 Oct 22 14:59 php.conf
-rw-r--r--. 1 root root 1252 Sep  3 23:56 userdir.conf
-rw-r--r--  1 root root  370 Nov 13 16:16 vhost.conf
-rw-r--r--. 1 root root  653 Sep  3 23:56 welcome.conf

현재 vhost 파일이 두개이상 있습니다
README, autoindex.conf, userdir.conf, welcome.conf외에 다른 vhost 파일들을 확인하시고
lamp_ftp_completed.ipynb의 READE ME를 읽고 알맞게 고쳐주세요.


...


현재 /etc/httpd/conf.d/vhost.conf의 내용은 아래와 같습니다
<VirtualHost *:80>
    ServerAdmin webmaster@localhost
    ServerName ns.test.com
    DocumentRoot /home/heeyoung

    <Directory /home/heeyoung>
        AllowOverride None
        Options Indexes FollowSymLinks
        Require all granted
    </Directory>

    ErrorLog /var/log/httpd/test_error.log
    CustomLog /var/log/httpd/test_access.log combined
</VirtualHost>
<VirtualHost *:80>
    ServerAdmin webmaster@localhost
    ServerName wp.test.com
    DocumentRoot "/home/wp/"

    <Directory "/home/wp/">
        AllowOverride None
        Options Indexes FollowSymLinks
        Require all granted
    </Directory>

</VirtualHost>



(1) vhost 추가하기, (0) vhost 파일 수정 종료 1
ServerName을 입력하세요. 주의!! 나중에 dns를 만들때 도메인 네임과 일치시키지 않으면 문제발생 (예: wp.test.com):  wpp.test.com



현재 /etc/httpd/conf.d/vhost.conf의 내용은 아래와 같습니다
<VirtualHost *:80>
    ServerAdmin webmaster@localhost
    ServerName ns.test.com
    DocumentRoot /home/heeyoung

    <Directory /home/heeyoung>
        AllowOverride None
        Options Indexes FollowSymLinks
        Require all granted
    </Directory>

    ErrorLog /var/log/httpd/test_error.log
    CustomLog /var/log/httpd/test_access.log combined
</VirtualHost>
<VirtualHost *:80>
    ServerAdmin webmaster@localhost
    ServerName wp.test.com
    DocumentRoot "/home/wp/"

    <Directory "/home/wp/">
        AllowOverride None
        Options Indexes FollowSymLinks
        Require all granted
    </Directory>

</VirtualHost>
<VirtualHost *:80>
    ServerAdmin webmaster@localhost
    ServerName wpp.test.com
    DocumentRoot "/home/wp/"

    <Directory "/home/wp/">
        AllowOverride None
        Options Indexes FollowSymLinks
        Require all granted
    </Directory>

</VirtualHost>



(1) vhost 추가하기, (0) vhost 파일 수정 종료 0


 /etc/httpd/conf.d/vhost.conf 생성 완료 및 httpd 및 named 재시작


